In [1]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input

In [2]:
from tensorflow.keras.models import Model

In [3]:
def conv_block(inputs, num_filters):
    x=Conv2D(num_filters, 3, padding="same")(inputs)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    
    x=Conv2D(num_filters, 3, padding="same")(inputs)
    x=BatchNormalization()(x)
    x=Activation("relu")(x)
    
    return x

In [4]:
def encoder_block(inputs, num_filters):
    x=conv_block(inputs, num_filters)
    p=MaxPool2D((2,2))(x)
    
    return x, p

In [5]:
def decoder_block(inputs, skip_features, num_filters):
    x=Conv2DTranspose(num_filters, (2,2), strides=2, padding="same")(inputs)
    x=Concatenate()([x, skip_features])
    x=conv_block(x, num_filters)
    return x

In [6]:
def build_unet(input_shape):
    inputs = Input(input_shape)
    
    """Encoder"""
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    
    """Bridge"""
    b1=conv_block(p4, 1024)
    
    """Decoder"""
    d1=decoder_block(b1, s4, 512)
    d2=decoder_block(d1, s3, 256)
    d3=decoder_block(d2, s2, 128)
    d4=decoder_block(d3, s1, 64)
    
    """Output"""
    outputs=Conv2D(1, (1,1), padding="same", activation="sigmoid")(d4)
    
    model=Model(inputs, outputs, name="U-Net")
    return model
if __name__== "__main__":
    input_shape= (512, 512, 3)
    model=build_unet(input_shape)
    model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 512, 512, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 512, 512, 64) 0           batch_normalization_1[0][0]      
______________________________________________________________________________________________